# Fake News Detection - Kaggle

https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset?select=True.csv

TOC:

* [processing](#proc)
* [neural network](#nn)
---

In [34]:
# Install libraries
#import sys
#!{sys.executable} -m pip install tensorflow
#!{sys.executable} -m pip install keras
#!{sys.executable} -m pip install xgboost
#!brew install libomp

Updating Homebrew...
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
acl2            cgl             empty           mariadb@10.4    staticcheck
borgbackup      charge          go@1.14         microplane      terraform@0.12
castget         datasette       gostatic        pandocomatic    usb.ids
cbc             eleventy        kondo           sqlite-utils    wownero
==> Updated Formulae
Updated 475 formulae.
==> Deleted Formulae
wpscan

==> Downloading https://homebrew.bintray.com/bottles/libomp-10.0.1.catalina.bott
==> Downloading from https://d29vzk4ow07wi7.cloudfront.net/8545af588b2b210708b56
######################################################################## 100.0%
==> Pouring libomp-10.0.1.catalina.bottle.tar.gz
🍺  /usr/local/Cellar/libomp/10.0.1: 9 files, 1.3MB


In [38]:
# Import libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn import metrics

# Get user functions
%run '/Users/zxs/Documents/code/nlp/zxs.py'

In [3]:
# Load data
fake = pd.read_csv('/Users/zxs/Documents/data/fake.csv')
real = pd.read_csv('/Users/zxs/Documents/data/true.csv')

In [4]:
%%time

# Assign labels
fake['label'] = 0
real['label'] = 1

# Combine
data = pd.concat([fake, real])
data['all'] = data['title'] + ' ' + data['text']

# Convert dt
data['date'] = pd.to_datetime(data['date'], errors = 'coerce')

# Apply function to parse text
data['parsed'] = data['all'].apply(parse)

In [5]:
data.head()

,title,text,subject,date,label,all,parsed
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,2017-12-31,0,Donald Trump Sends Out Embarrassing New Year’...,donald trump sends out embarrassing new year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,2017-12-31,0,Drunk Bragging Trump Staffer Started Russian ...,drunk bragging trump staffer started russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,2017-12-30,0,Sheriff David Clarke Becomes An Internet Joke...,sheriff david clarke becomes an internet joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,2017-12-29,0,Trump Is So Obsessed He Even Has Obama’s Name...,trump is so obsessed he even has obama’s name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,2017-12-25,0,Pope Francis Just Called Out Donald Trump Dur...,pope francis just called out donald trump dur...


In [17]:
data.isnull().sum()

title       0
text        0
subject     0
date       10
label       0
all         0
parsed      0
dtype: int64

In [25]:
# Initialize vec
cv = CountVectorizer()
x = cv.fit_transform(data['parsed'])

# Split data for training / testing
xtrain, xval, ytain, yval = train_test_split(x, data['label'], test_size = .3, random_state = 100)

In [29]:
# Function to train a given model
def training_evaluation(model):
    
    m = model
    m.fit(xtrain, ytrain)
    
    preds = m.predict(xval)
    
    return print('Results for model: ', metrics.accuracy_score(preds, yval)

In [39]:
training_evaluation(xgb.XGBClassifier())

0.9970304380103935